# Altering a Spyglass DataJoint table


This notebook was used to do the following modifications to Loren Frank Lab's Spyglass database:

1. Move table LFPElectrodeGroup from lfp_v1 schema to new schema called lfp_electrode
2. Move table ImportedLFPV1 from lfp_v1 schema to new schema called lfp_imported
3. Remove empty tables in the lfp-related schema that are not associated with any code and create issues when plotting ERDs

See https://github.com/LorenFrankLab/spyglass/pull/594


## Prepare local test database using backup of production database

ssh into a frank lab machine

```bash
ssh rly@typhoon.cin.ucsf.edu -p XXXX
```

Zip up the latest database backup and place in your home directory on the frank lab remote storage.
Change the dates in the file names as appropriate

```bash
tar -cvf lmf-db-20230722.tar.gz /stelmo/mysql-backup/lmf-db-20230722/
```

NOTE: we may only really need the mysql-all.sql file but it's good to have a full backup handy

Copy the zipped up database backup to your local machine

NOTE: this is a large file >11 GB

```bash
scp -P XXXX rly@typhoon.cin.ucsf.edu:lmf-db-20230722.tar.gz ~/Downloads/
cd ~/Downloads
tar -xzvf lmf-db-20230720.tar.gz
```

This will unzip into a stelmo folder in ~/Downloads

Start a clean datajoint mysql database in docker locally on Ryan's laptop, that involves:

```bash
cd ~/Documents/NWB/datajoint-mysql-docker
docker-compose up -d
```

Load the database backup into the local empty database.
This will take ~20 minutes depending on the size of the backup and speed of the computer...

```bash
cat ~/Downloads/stelmo/mysql-backup/lmf-db-20230722/mysql-all.sql | docker exec -i dj mysql -u root --password=tutorial -h 127.0.0.1
```


# Examine the backup file for references to the tables being altered

For LFPElectrodeGroup (encoded as l_f_p_electrode_group in mysql tables):

Search for all instances of the table name being altered in mysql-all.sql

```bash
grep l_f_p_electrode_group ~/Downloads/stelmo/mysql-backup/lmf-db-20230722/mysql-all.sql > out.txt
```

Take note of which tables these are in. for example:

- l_f_p_band_selection**l_f_p_band_electrode_ibfk_2 is a foreign key from LFPBandSelection.\_LFPBandElectrode that references
  `` `lfp_v1`.`l_f_p_electrode_group**l_f_p_electrode` ``
- Similarly for \_imported_l_f_p_ibfk_2 and \_imported_l_f_p_v1_ibfk_2
- l_f_p_band_selection**l_f_p_band_electrode_ibfk_2 is a foreign key from LFPBandSelection.\_LFPBandElectrode that references
  `l_f_p_electrode_group**l_f_p_electrode`
- l_f_p_selection_ibfk_1 is a foreign key from LFPSelection that references `l_f_p_electrode_group`

Ignore all logs and innodb_table_stats.

Use `grep -A 20 -B 20` to get context around each match

Also search the spyglass codebase for references to this table

- imported in `lfp/v1/__init__.py`
- defined in `lfp/v1/lfp.py`
- has a part table
- used as primary key in LFPSelection in `lfp/v1/lfp.py`
- used in make function of LFPV1
- used as primary key in ImportedLFPV1 in `lfp/v1/lfp.py`
- used as primary key in LFPBandSelection.LFPBandElectrode in `lfp_band/v1/lfp_band.py` and in code

- the `lfp/v1/lfp.py::ImportedLFP` table is not in the codebase and therefore should not exist in the database. It is empty and seems to be causing issues with plotting ERDs. It must have been created once from a local or temporary version of the codebase and then the code was updated.
- LFPOutput.ImportedLFP still has a foreign key to ImportedLFP. This should be a foreign key to ImportedLFPV1.

The ImportedLFPV1 table is being moved and renamed. LFPOutput.ImportedLFPV1 is also being renamed.

- it is currently empty.
- ImportedLFP is also empty.
- LFPOutput.ImportedLFP is also empty.

Drop all these tables and re-initialize with the PR.

Need to find all the foreign keys to these tables and drop those keys (or tables) before dropping the above tables.
In this order:

- Drop empty LFPMerge.ImportedLFPV1
- Drop empty LFPOutput.ImportedLFP
- Drop empty ImportedLFPV1
- Drop empty ImportedLFP
  ...

Delete the local docker volume and reload the backup database as needed to nail down the sequence of steps that you will perform on the production database.


Make sure your repo is on the master branch


Configure a connection to the local database for a dry run - skip this cell if running on frank lab servers on the production database


In [ ]:
import datajoint as dj
import os
from pathlib import Path

# set dirs
base_dir = Path(
    "/Users/rly/Documents/NWB/spyglass-workspace"
)  # change this to your desired directory
raw_dir = base_dir / "raw"
analysis_dir = base_dir / "analysis"
recording_dir = base_dir / "recording"
sorting_dir = base_dir / "sorting"
waveforms_dir = base_dir / "waveforms"
tmp_dir = base_dir / "tmp"
kachery_cloud_dir = base_dir / ".kachery_cloud"

os.makedirs(raw_dir, exist_ok=True)
os.makedirs(analysis_dir, exist_ok=True)
os.makedirs(recording_dir, exist_ok=True)
os.makedirs(sorting_dir, exist_ok=True)
os.makedirs(waveforms_dir, exist_ok=True)
os.makedirs(tmp_dir, exist_ok=True)
os.makedirs(kachery_cloud_dir, exist_ok=True)

# set dj config
dj.config["database.host"] = "localhost"
dj.config["database.user"] = "root"
dj.config["database.password"] = "tutorial"
dj.config["database.port"] = 3306
dj.config["stores"] = {
    "raw": {
        "protocol": "file",
        "location": str(raw_dir),
        "stage": str(raw_dir),
    },
    "analysis": {
        "protocol": "file",
        "location": str(analysis_dir),
        "stage": str(analysis_dir),
    },
}
dj.config["enable_python_native_blobs"] = True

# set env vars
os.environ["SPYGLASS_BASE_DIR"] = str(base_dir)
os.environ["SPYGLASS_RECORDING_DIR"] = str(recording_dir)
os.environ["SPYGLASS_SORTING_DIR"] = str(sorting_dir)
os.environ["SPYGLASS_WAVEFORMS_DIR"] = str(waveforms_dir)
os.environ["SPYGLASS_TEMP_DIR"] = str(tmp_dir)
os.environ["KACHERY_CLOUD_DIR"] = str(kachery_cloud_dir)
os.environ["DJ_SUPPORT_FILEPATH_MANAGEMENT"] = "TRUE"

## Local Server

Start here instead of above if running on the Frank Lab servers on the production database


In [ ]:
import datajoint as dj
import spyglass as sg

sg.config["prepopulate"] = False

import spyglass.common as sgc
import spyglass.data_import as sgdi
import spyglass.lfp as lfp
import spyglass.lfp.v1 as lfp_v1
import spyglass.lfp.analysis.v1.lfp_band as lfp_band

# Plot 

Plot ERDs of the tables being altered to see what tables depend on these tables


In [ ]:
dj.ERD(lfp_v1.LFPElectrodeGroup) + 2

In [ ]:
dj.ERD(lfp_v1.ImportedLFP) + 2

# Start with tasks #2 and #3


# Confirm that the tables being dropped are empty

- If the table exists, just print the contents, e.g., `lfp.LFPOutput.ImportedLFPV1`
- If the table does not exist, use a MySQL database browser, like TablePlus for macOS.


In [ ]:
lfp.LFPOutput.ImportedLFPV1()

The `lfp_v1.l_f_p_output` table shouldn't even exist anymore. There are no references in the codebase to it anymore. It should have been replaced with the `lfp_merge.l_f_p_output` table. But the data in the two tables differ... Probably `lfp_v1.l_f_p_output` is old. It has only 80 rows. Ignore this for now.


# Drop tables starting with those that no other tables depend upon

Carefully, one at a time...

The expected output if the command passes is `<pymysql.cursors.Cursor at ...>`


In [ ]:
conn = dj.conn()
conn.query("USE `lfp_merge`")
conn.query(
    "DROP TABLE `lfp_merge`.`l_f_p_output__imported_l_f_p_v1`"
)  # can't use datajoint to drop this because it's a part table

In [ ]:
conn = dj.conn()
conn.query("USE `lfp_v1`")
conn.query(
    "DROP TABLE `lfp_v1`.`l_f_p_output__imported_l_f_p`"
)  # can't use datajoint to drop this because it's a part table and also doesn't exist in the code

In [ ]:
conn = dj.conn()
conn.query("USE `lfp_v1`")
conn.query(
    "DROP TABLE `lfp_v1`.`_imported_l_f_p_v1`"
)  # can't use datajoint to drop this because it doesn't exist in the code

In [ ]:
conn = dj.conn()
conn.query("USE `lfp_v1`")
conn.query(
    "DROP TABLE `lfp_v1`.`_imported_l_f_p`"
)  # can't use datajoint to drop this because it doesn't exist in the code

# Task 1. Rename 

Rename the tables while maintaining foreign key references from other tables to these tables


In [ ]:
dj.schema("lfp_electrode")  # create a new database

In [ ]:
conn = dj.conn()
conn.query("USE `lfp_v1`")
conn.query(
    "ALTER TABLE `lfp_v1`.`l_f_p_electrode_group` RENAME `lfp_electrode`.`l_f_p_electrode_group`"
)

In [ ]:
conn = dj.conn()
conn.query("USE `lfp_v1`")
conn.query(
    "ALTER TABLE `lfp_v1`.`l_f_p_electrode_group__l_f_p_electrode` RENAME `lfp_electrode`.`l_f_p_electrode_group__l_f_p_electrode`"
)

# Verify the changes

Dump individual databases to a file and search for lingering references to `lfp_v1.l_f_p_electrode_group` or `lfp_v1.l_f_p_electrode_group__l_f_p_electrode`. Make sure they have been updated.

```
docker exec -i dj mysqldump -u root --password=tutorial -h 127.0.0.1 --databases lfp_v1 > dump_lfp_v1.sql
docker exec -i dj mysqldump -u root --password=tutorial -h 127.0.0.1 --databases lfp_band_v1 > dump_lfp_band_v1.sql
```

You might also just be able to run:

```
SELECT * FROM information_schema.KEY_COLUMN_USAGE WHERE REFERENCED_TABLE_NAME = 'l_f_p_electrode_group'
SELECT * FROM information_schema.KEY_COLUMN_USAGE WHERE REFERENCED_TABLE_NAME = 'l_f_p_electrode_group__l_f_p_electrode'
```

If time permits, run through the steps of this notebook again to make sure it is smooth and foolproof before running this on the production database.


# Then switch git branches to the PR with the new changes


# Restart the Jupyter kernel. Confirm things work. Connect to the local database again.


In [1]:
import datajoint as dj
import os
from pathlib import Path

# set dirs
base_dir = Path(
    "/Users/rly/Documents/NWB/spyglass-workspace"
)  # change this to your desired directory
raw_dir = base_dir / "raw"
analysis_dir = base_dir / "analysis"
recording_dir = base_dir / "recording"
sorting_dir = base_dir / "sorting"
waveforms_dir = base_dir / "waveforms"
tmp_dir = base_dir / "tmp"
kachery_cloud_dir = base_dir / ".kachery_cloud"

os.makedirs(raw_dir, exist_ok=True)
os.makedirs(analysis_dir, exist_ok=True)
os.makedirs(recording_dir, exist_ok=True)
os.makedirs(sorting_dir, exist_ok=True)
os.makedirs(waveforms_dir, exist_ok=True)
os.makedirs(tmp_dir, exist_ok=True)
os.makedirs(kachery_cloud_dir, exist_ok=True)

# set dj config
dj.config["database.host"] = "localhost"
dj.config["database.user"] = "root"
dj.config["database.password"] = "tutorial"
dj.config["database.port"] = 3306
dj.config["stores"] = {
    "raw": {
        "protocol": "file",
        "location": str(raw_dir),
        "stage": str(raw_dir),
    },
    "analysis": {
        "protocol": "file",
        "location": str(analysis_dir),
        "stage": str(analysis_dir),
    },
}
dj.config["enable_python_native_blobs"] = True

# set env vars
os.environ["SPYGLASS_BASE_DIR"] = str(base_dir)
os.environ["SPYGLASS_RECORDING_DIR"] = str(recording_dir)
os.environ["SPYGLASS_SORTING_DIR"] = str(sorting_dir)
os.environ["SPYGLASS_WAVEFORMS_DIR"] = str(waveforms_dir)
os.environ["SPYGLASS_TEMP_DIR"] = str(tmp_dir)
os.environ["KACHERY_CLOUD_DIR"] = str(kachery_cloud_dir)
os.environ["DJ_SUPPORT_FILEPATH_MANAGEMENT"] = "TRUE"

In [ ]:
import spyglass as sg

sg.config["prepopulate"] = False

import spyglass.common as sgc # noqa: F811
import spyglass.data_import as sgdi  # noqa: F811
import spyglass.lfp as lfp
import spyglass.lfp.v1 as lfp_v1
import spyglass.lfp.analysis.v1.lfp_band as lfp_band  # noqa: F811

[2023-07-23 23:56:50,225][INFO]: Connecting root@localhost:3306
[2023-07-23 23:56:50,363][INFO]: Connected root@localhost:3306


In [3]:
lfp.LFPElectrodeGroup()

nwb_file_name name of the NWB file,lfp_electrode_group_name the name of this group of electrodes
arthur20220315_.nwb,all_tets_arthur
arthur20220316_.nwb,all_tets_arthur
arthur20220317_.nwb,all_tets_arthur
arthur20220318_.nwb,all_tets_arthur
arthur20220319_.nwb,all_tets_arthur
arthur20220320_.nwb,all_tets_arthur
arthur20220321_.nwb,all_tets_arthur
arthur20220322_.nwb,all_tets_arthur
arthur20220323_.nwb,all_tets_arthur
arthur20220324_.nwb,all_tets_arthur


In [4]:
lfp_v1.LFPSelection()

nwb_file_name name of the NWB file,lfp_electrode_group_name the name of this group of electrodes,target_interval_list_name descriptive name of this interval list,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter
arthur20220315_.nwb,all_tets_arthur,r1_r2,LFP 0-400 Hz,30000
arthur20220315_.nwb,all_tets_arthur,r2_r3,LFP 0-400 Hz,30000
arthur20220316_.nwb,all_tets_arthur,r1_r2,LFP 0-400 Hz,30000
arthur20220316_.nwb,all_tets_arthur,r2_r3,LFP 0-400 Hz,30000
arthur20220317_.nwb,all_tets_arthur,r1_r2,LFP 0-400 Hz,30000
arthur20220317_.nwb,all_tets_arthur,r2_r3,LFP 0-400 Hz,30000
arthur20220318_.nwb,all_tets_arthur,r1_r2,LFP 0-400 Hz,30000
arthur20220318_.nwb,all_tets_arthur,r2_r3,LFP 0-400 Hz,30000
arthur20220319_.nwb,all_tets_arthur,r2_r2,LFP 0-400 Hz,30000
arthur20220319_.nwb,all_tets_arthur,r2_r3,LFP 0-400 Hz,30000


In [7]:
from spyglass.lfp.lfp_electrode import LFPElectrodeGroup

LFPElectrodeGroup()

nwb_file_name name of the NWB file,lfp_electrode_group_name the name of this group of electrodes
arthur20220315_.nwb,all_tets_arthur
arthur20220316_.nwb,all_tets_arthur
arthur20220317_.nwb,all_tets_arthur
arthur20220318_.nwb,all_tets_arthur
arthur20220319_.nwb,all_tets_arthur
arthur20220320_.nwb,all_tets_arthur
arthur20220321_.nwb,all_tets_arthur
arthur20220322_.nwb,all_tets_arthur
arthur20220323_.nwb,all_tets_arthur
arthur20220324_.nwb,all_tets_arthur


# Run this on the production database on the frank lab servers

- Log in to the frank lab server
- Reinstall the master branch of spyglass

```
ssh rly@typhoon.cin.ucsf.edu -p XXXX
cd spyglass
# remove the existing spyglass repo if it exists
mamba remove --name spyglass --all --yes
git status
# make sure i am on the master branch
git pull
mamba env create -f environment.yml --verbose  # this can take like 30 minutes
mamba activate spyglass
pip install -e .
```

- Copy this notebook there
- Start jupyter
- Run through all the above steps EXCEPT do not configure datajoint to connect to a local database. Datajoint is already configured to connect to the Frank Lab spyglass production database (at least on Ryan's account)


# A scrapped idea - changing a primary key of a table that is used by lots of downstream tables

2. Copy all entries from common_ephys.LFPSelection to new lfp_electrode.LFPElectrodeGroup and set the "lfp_electrode_group_name" values to "full_session_full_channel_set"
3. Add a column in common_ephys.LFP that is a foreign key to new lfp_electrode.LFPElectrodeGroup with the values equal to the new entries in lfp_electrode.LFPElectrodeGroup
4. Change the primary key of common_ephys.LFP to new lfp_electrode.LFPElectrodeGroup (this might involve adding a new primary key and then removing the old one).

- If changing the primary key requires manually adding the new "lfp_electrode_group_name" to every table downstream of common_ephys.LFP, then abort mission.
- To add a new primary key, we would need to drop the foreign key to the primary key of common_ephys.LFP (recursively) on every downstream table, some of which may be primary keys themselves. This would require dropping basically reference to common_ephys.LFP and every reference to a table that references that, etc.

### ChatGPT says:

If you encounter the "Foreign key constraint is incorrectly formed" error when trying to drop the primary key, it might be because there are foreign key constraints referencing the primary key you're attempting to drop.

To add a new column to an existing primary key without encountering this issue, you can follow these steps:

1. **Drop Foreign Key Constraints**:
   You need to drop any foreign key constraints that reference the existing primary key before modifying it. Identify the foreign key constraints that are using the primary key you want to modify, and drop them using the `ALTER TABLE` statement. For example:

   ```sql
   -- Drop foreign key constraints that reference the primary key
   ALTER TABLE referencing_table1 DROP FOREIGN KEY constraint_name1;
   ALTER TABLE referencing_table2 DROP FOREIGN KEY constraint_name2;
   -- ... (repeat for each referencing table)
   ```

   Replace `referencing_table1`, `referencing_table2`, etc., with the names of the tables that have foreign key constraints, and `constraint_name1`, `constraint_name2`, etc., with the actual names of the foreign key constraints.

2. **Drop Primary Key Constraint**:
   Once all foreign key constraints have been dropped, you can drop the existing primary key constraint:

   ```sql
   ALTER TABLE users DROP PRIMARY KEY;
   ```

   Replace `users` with the name of your table.

3. **Add the New Column and Primary Key**:
   Now that the primary key constraint is removed, you can add the new column and designate it as the primary key:

   ```sql
   ALTER TABLE users
   ADD COLUMN new_id INT AUTO_INCREMENT PRIMARY KEY;
   ```

   Replace `users` with the name of your table and `new_id` with the name of the new column.

4. **Recreate Foreign Key Constraints**:
   After successfully adding the new primary key column, you can re-create the foreign key constraints that were previously dropped:

   ```sql
   ALTER TABLE referencing_table1
   ADD CONSTRAINT constraint_name1 FOREIGN KEY (foreign_key_column1) REFERENCES users(new_id);

   ALTER TABLE referencing_table2
   ADD CONSTRAINT constraint_name2 FOREIGN KEY (foreign_key_column2) REFERENCES users(new_id);

   -- ... (repeat for each referencing table)
   ```

   Replace `referencing_table1`, `referencing_table2`, etc., with the names of the tables that have foreign key constraints, `constraint_name1`, `constraint_name2`, etc., with the desired names of the foreign key constraints, and `foreign_key_column1`, `foreign_key_column2`, etc., with the columns in the referencing tables that reference the new primary key column.

Please be cautious when modifying primary keys and foreign key constraints, as they are critical to maintaining data integrity. Always have proper backups and thoroughly test any changes in a safe environment before applying them to production data.


# Old cells from previous database surgery


# Alter DataAcquisitionDevice and create DataAcquisitionDeviceSystem and DataAcquisitionDeviceAmplifier tables


In [ ]:
# inspect values
sgc.DataAcquisitionDevice()

In [ ]:
# store the values in memory
res = sgc.DataAcquisitionDevice.fetch()
res

In [ ]:
# rename the fields
res.dtype.names = [
    "data_acquisition_device_name",
    "data_acquisition_device_system",
    "data_acquisition_device_amplifier",
    "adc_circuit",
]
res

In [ ]:
# drop DataAcquisitionDevice
# this will also drop sgc.SessionDataAcquisitionDevice() but this should not exist anyway (it may have been created during testing)
sgc.DataAcquisitionDevice().drop()

In [ ]:
# these new tables may already exist and have dummy data from testing
# temporarily make a class for them and then drop the tables
schema = dj.schema("common_device")


@schema
class DataAcquisitionDeviceSystem(dj.Manual):
    pass


@schema
class DataAcquisitionDeviceAmplifier(dj.Manual):
    pass


DataAcquisitionDeviceSystem.drop()
DataAcquisitionDeviceAmplifier.drop()

In [ ]:
# load new class definitions
schema = dj.schema("common_device")


@schema
class DataAcquisitionDeviceSystem(dj.Manual):
    definition = """
    # Known data acquisition device system names.
    data_acquisition_device_system: varchar(80)
    ---
    """


@schema
class DataAcquisitionDeviceAmplifier(dj.Manual):
    definition = """
    # Known data acquisition device amplifier names.
    data_acquisition_device_amplifier: varchar(80)
    ---
    """


@schema
class DataAcquisitionDevice(dj.Manual):
    definition = """
    data_acquisition_device_name: varchar(80)
    ---
    -> DataAcquisitionDeviceSystem
    -> DataAcquisitionDeviceAmplifier
    adc_circuit = NULL: varchar(2000)
    """

In [ ]:
# populate DataAcquisitionDeviceSystem
for name in res["data_acquisition_device_system"]:
    DataAcquisitionDeviceSystem.insert1(
        {"data_acquisition_device_system": name}, skip_duplicates=True
    )
DataAcquisitionDeviceSystem()

In [ ]:
# populate DataAcquisitionDeviceAmplifier
for name in res["data_acquisition_device_amplifier"]:
    DataAcquisitionDeviceAmplifier.insert1(
        {"data_acquisition_device_amplifier": name}, skip_duplicates=True
    )
DataAcquisitionDeviceAmplifier()

In [ ]:
# populate DataAcquisitionDevice
DataAcquisitionDevice.insert(res)
DataAcquisitionDevice()

# Create new ProbeType table


In [ ]:
# these new tables may already exist and have dummy data from testing
# temporarily make a class for them and then drop the tables
schema = dj.schema("common_device")


@schema
class ProbeType(dj.Manual):
    definition = """
    # Type/category of probe, e.g., Neuropixels 1.0 or NeuroNexus X-Y-Z, regardless of configuration.
    # This is a controlled vocabulary of probe type names.
    # This is separated from Probe because probes like the Neuropixels 1.0 can have different dynamic configurations,
    # e.g. channel maps.
    probe_type: varchar(80)
    ---
    probe_description: varchar(2000)               # description of this probe
    manufacturer = "": varchar(200)                # manufacturer of this probe
    num_shanks: int                                # number of shanks on this probe
    """


ProbeType.drop()

In [ ]:
# load new class definition
schema = dj.schema("common_device")


@schema
class ProbeType(dj.Manual):
    definition = """
    # Type/category of probe, e.g., Neuropixels 1.0 or NeuroNexus X-Y-Z, regardless of configuration.
    # This is a controlled vocabulary of probe type names.
    # This is separated from Probe because probes like the Neuropixels 1.0 can have different dynamic configurations,
    # e.g. channel maps.
    probe_type: varchar(80)
    ---
    probe_description: varchar(2000)               # description of this probe
    manufacturer = "": varchar(200)                # manufacturer of this probe
    num_shanks: int                                # number of shanks on this probe
    """

In [ ]:
# populate ProbeType from Probe
res = sgc.Probe.fetch()
for row in res:
    ProbeType.insert1(
        {
            "probe_type": row["probe_type"],
            "probe_description": row["probe_description"],
            "num_shanks": row["num_shanks"],
        }
    )
ProbeType()

# Alter primary key of Probe table


In [ ]:
# confirm that only a few tables are affected by these changes
# only Probe.Shank and ElectrodeGroup should use Probe
dj.ERD(sgc.Probe) + 1

In [ ]:
# only Probe.Electrode should use Probe.Shank
dj.ERD(sgc.Probe.Shank) + 1

In [ ]:
# only Electrode should use Probe.Electrode
dj.ERD(sgc.Probe.Electrode) + 1

In [ ]:
conn = dj.conn()
conn.query("USE `common_device`")
conn.query(
    "ALTER TABLE `probe` CHANGE `probe_type` `probe_id` varchar(80) NOT NULL"
)
conn.query(
    "ALTER TABLE `probe__shank` CHANGE `probe_type` `probe_id` varchar(80) NOT NULL"
)
conn.query(
    "ALTER TABLE `probe__electrode` CHANGE `probe_type` `probe_id` varchar(80) NOT NULL"
)
conn.query("USE `common_ephys`")
conn.query(
    "ALTER TABLE `_electrode_group` CHANGE `probe_type` `probe_id` varchar(80) DEFAULT NULL"
)
conn.query(
    "ALTER TABLE `_electrode_group` DROP KEY `probe_type`, ADD KEY `probe_id` (`probe_id`)"
)
conn.query(
    "ALTER TABLE `_electrode` CHANGE `probe_type` `probe_id` varchar(80) DEFAULT NULL"
)
conn.query(
    "ALTER TABLE `_electrode` DROP KEY `probe_type`, ADD KEY `probe_id` (`probe_id`,`probe_shank`,`probe_electrode`)"
)

1. Update spyglass code to reflect the name change (`git checkout file_based_mods`)
1. Restart kernel
1. Dump databases common_device and common_ephys to file to make sure the name change cascaded correctly
1. `docker exec -i dj mysqldump -u root --password=tutorial -h 127.0.0.1 --databases common_device common_ephys > dump.sql`
1. Run first cell
1. Print tables in spyglass to make sure the name change worked

In [ ]:
# verify the change
sgc.Probe()

In [ ]:
sgc.Probe.Shank()

In [ ]:
sgc.Probe.Electrode()

In [ ]:
sgc.ElectrodeGroup()

In [ ]:
sgc.Electrode()

# Alter columns of Probe table


In [ ]:
# load new class definition
schema = dj.schema("common_device")
from spyglass.common import DataAcquisitionDevice


@schema
class Probe(dj.Manual):
    definition = """
    # A configuration of a ProbeType. For most probe types, there is only one configuration, and that configuration
    # should always be used. For Neuropixels probes, the specific channel map (which electrodes are used,
    # where are they, and in what order) can differ between users and sessions, and each configuration should have a
    # different ProbeType.
    probe_id: varchar(80)     # a unique ID for this probe and dynamic configuration
    ---
    -> ProbeType              # the type of probe, selected from a controlled list of probe types
    -> [nullable] DataAcquisitionDevice  # the data acquisition device used with this Probe
    contact_side_numbering: enum("True", "False")  # if True, then electrode contacts are facing you when numbering them
    """

In [ ]:
conn = dj.conn()
conn.query("USE `common_device`")
conn.query(
    "ALTER TABLE `probe` "
    "DROP `probe_description`, "
    "DROP `num_shanks`, "
    "ADD `probe_type` varchar(80) NOT NULL DEFAULT '' COMMENT '' AFTER `probe_id`, "
    "ADD `data_acquisition_device_name` varchar(80) COMMENT '' AFTER `probe_type`"
)

In [ ]:
conn = dj.conn()
conn.query("USE `common_device`")
conn.query(
    "UPDATE `probe` SET `probe_type` = `probe_id` where `probe_type` = ''"
)

In [ ]:
conn = dj.conn()
conn.query("USE `common_device`")
conn.query(
    "ALTER TABLE `probe` "
    "ADD FOREIGN KEY (`probe_type`) REFERENCES `common_device`.`probe_type` (`probe_type`) ON UPDATE CASCADE ON DELETE RESTRICT, "
    "ADD FOREIGN KEY (`data_acquisition_device_name`) REFERENCES `common_device`.`data_acquisition_device` (`data_acquisition_device_name`) ON UPDATE CASCADE ON DELETE RESTRICT"
)

1. Restart kernel
1. Dump databases common_device and common_ephys to file to make sure the name change cascaded correctly
1. `docker exec -i dj mysqldump -u root --password=tutorial -h 127.0.0.1 --databases common_device > dump.sql`
1. Run cell 1

In [ ]:
# verify this is correct
sgc.Probe()

In [ ]:
# verify the join works
sgc.Probe() * sgc.ProbeType()

# Move ExperimenterList data to Session.Experimenter


In [ ]:
# access the old ExperimenterList table
schema = dj.schema("common_session")


@schema
class ExperimenterList(dj.Imported):
    definition = """
    -> Session
    """

    class Experimenter(dj.Part):
        definition = """
        -> ExperimenterList
        -> LabMember
        """

In [ ]:
res = ExperimenterList.Experimenter()
res

In [ ]:
sgc.Session.Experimenter.insert(res)

In [ ]:
sgc.Session.Experimenter()

In [ ]:
ExperimenterList.drop()